# (Prototype) COVID Dashboard Map

This is a prototype COVID Dashboard for testing displaying COVID data on maps (initially testing using the folium library).

In [2]:
# This forces a reload of any external library file if it changes.  
# Useful when developing external libraries since otherwise Jupyter 
# will not re-import any library without restarting the python kernel.

%load_ext autoreload
%autoreload 2

In [3]:
import os
import pandas as pd
import numpy as np
import ipywidgets as widgets

# Import COVID IO routines from external python libraries
import COVIDlib.data_IO as COVID_IO
import COVIDlib.dashboard_IO as COVID_Dash
import COVIDlib.dashboard_maps as COVID_Maps

## Define variables of interest below
data_dir = 'our_data/'    # Data directory for the COVID datafiles

In [4]:
# Load all the dataframes into memory

# Retrieve John Hopkins dataframes and add "rates" of deaths/infections
(JH_state_df, JH_cnty_df) = COVID_IO.PtoCDRDataFrames()
JH_state_df = COVID_Dash.cleanJHdata(JH_state_df)
JH_cnty_df = COVID_Dash.cleanJHdata(JH_cnty_df)

# Construct dictionary of FIPS values by placename
FIPSd = COVID_Dash.build_fipsdict(JH_cnty_df, JH_state_df)

# Retrieve Apple Mobility Dataframes
(aapl_cnty_df, aapl_state_df) = COVID_IO.PtoAAPLMobilityDataFrames()
COVID_Dash.cleanAAPLdata(aapl_cnty_df)
COVID_Dash.cleanAAPLdata(aapl_state_df)
# Retrieve Google Mobility Dataframes
(goog_cnty_df, goog_state_df) = COVID_IO.PtoGOOGMobilityDataFrames()
# Load mobility variable descriptions
mobilityVarDict = COVID_Dash.BuildMobilityVarDict()

# Retrieve IMHE Dataframes
(summary_df, hospitalization_df) = COVID_IO.PtoIMHEDataFrames()
IMHEVarDict = COVID_Dash.BuildIMHEHospitalizationVarDict()

## Implementing Maps using iPyLeaflet 

In [5]:
# ipyleaflet map testing below
# The reason for using ipyleaflet is that it directly supports widgets
# Luke's to do: get JSON data to work, add choropleth

In [6]:
# Pick the data column to plot
colname = 'dConfirmedRate'

# Plot the US Map with State-level data
test_map = COVID_Maps.build_us_statesmap(JH_state_df, colname)

display(test_map)

Map(center=[38.0, -93.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [7]:
# Example of contents of state GEOJSON data structure (WHICH DOES WORK)
# {'type': 'FeatureCollection',
#  'features': [{'type': 'Feature',
#    'id': 'AL',
#    'properties': {'name': 'Alabama'},
#    'geometry': {'type': 'Polygon',
#     'coordinates': [[[-87.359296, 35.00118],
#       [-85.606675, 34.984749],
#       [-85.431413, 34.124869],
#       [-85.184951, 32.859696],
#       [-85.069935, 32.580372],
#       [-84.960397, 32.421541],

# Example of the 2018 US Census Bureau derived county boundaries (WHICH DOESN'T WORK WITHOUT MODIFICATION)
# {'type': 'FeatureCollection',
#  'name': 'cb_2018_us_county_20m',
#  'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4269'}},
#  'features': [{'type': 'Feature',
#    'properties': {'STATEFP': '37',
#     'COUNTYFP': '017',
#     'COUNTYNS': '01026336',
#     'AFFGEOID': '0500000US37017',
#     'GEOID': '37017',
#     'NAME': 'Bladen',
#     'LSAD': '06',
#     'ALAND': 2265887723,
#     'AWATER': 33010866},
#    'geometry': {'type': 'Polygon',
#     'coordinates': [[[-78.901998, 34.835268],
#       [-78.799597, 34.850857],
#       [-78.516123, 34.845919],
#       [-78.494705, 34.856182],
#       [-78.390748, 34.749463],
#       [-78.391978, 34.741265],
#       [-78.374363, 34.700722],
#       [-78.254441, 34.553595],
#       [-78.255468, 34.508614],

# I confirmed if I just loop through the list of dictionaries in 'features' and create an 'id' entry equal to the 
# 'GEOID' of that entry's 'properties' we can get plotting to work.

In [20]:
# Pick the data column to plot
colname = 'dConfirmedRate'

# Plot the US Map with State-level data
test_map2 = COVID_Maps.build_us_cntymap(JH_cnty_df, colname)

display(test_map2)

Map(center=[38.0, -93.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [18]:
JH_cnty_df.columns

Index(['FIPS', 'county', 'state', 'Lat', 'Long_', 'dates', 'Confirmed',
       'Deaths', 'Recovered', 'dConfirmed', 'd2Confirmed', 'dDeaths',
       'd2Deaths', 'PopEst2019', 'PopChg2019', 'postal', 'ConfirmedRate',
       'DeathRate', 'dConfirmedRate', 'dDeathsRate', 'd2ConfirmedRate',
       'd2DeathsRate', 'Active'],
      dtype='object')

In [21]:
# Still can't update the map above properly... the legend gets mangled and the overlay and 
# actual map disappears, sigh
COVID_Maps.update_us_cntymap(JH_cnty_df, 'Deaths', test_map2)